In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.ops import unary_union
from functools import cmp_to_key

from polygons import split_polygon, snake_sort, ns, sn, ew, we

In [2]:
quarters = gpd.read_file("../data/quarter_sections.geojson")
blocks1 = gpd.read_file("../data/blocks.json")
blocks2 = gpd.read_file("../data/39-14-27.json")
original_town = gpd.read_file("../data/original_town.json")

In [3]:
quarters.head()

,OBJECTID,TOWNSHIP,RANGE,SECTN,QUARTERSEC,IBL,FRACL,SHAPE_Length,SHAPE_Area,geometry
0,1,37,14,13,N.E.1/4,NIBL,1,7131.482171,2.781041e+06,"POLYGON ((-87.57419 41.70273, -87.56941 41.702..."
1,2,37,14,13,N.W.1/4,NIBL,1,10201.904596,6.816353e+06,"POLYGON ((-87.57982 41.70788, -87.58047 41.707..."
2,3,37,14,14,N.E.1/4,NIBL,0,10670.819542,7.116323e+06,"POLYGON ((-87.59411 41.70029, -87.59398 41.700..."
3,4,37,14,13,N.E.1/4,SIBL,1,10598.997285,7.021105e+06,"POLYGON ((-87.56442 41.70277, -87.56444 41.702..."
4,5,37,14,13,N.W.1/4,SIBL,0,10794.114317,6.416795e+06,"POLYGON ((-87.57419 41.70273, -87.57476 41.702..."


In [4]:
sect7 = unary_union(quarters.loc[(quarters['TOWNSHIP'] == 39)&(quarters['RANGE']==14)&(quarters['SECTN']=='7'), 'geometry'])

In [5]:
new_polygons = snake_sort(split_polygon(sect7, 8, 8).geoms, 8, "NE")

In [6]:
blocks3 = gpd.GeoDataFrame(data={'block': [i for i in range(1,65)]}, geometry=new_polygons)

In [7]:
blocks3.head()

,block,geometry
0,1,"POLYGON ((-87.66937 41.89604, -87.66729 41.896..."
1,2,"POLYGON ((-87.67187 41.89600, -87.66973 41.896..."
2,3,"POLYGON ((-87.67436 41.89596, -87.67340 41.895..."
3,4,"POLYGON ((-87.67686 41.89592, -87.67584 41.895..."
4,5,"POLYGON ((-87.67936 41.89588, -87.67706 41.895..."


In [8]:
blocks = gpd.GeoDataFrame(pd.concat([blocks1,blocks2,blocks3], ignore_index=True) )

In [9]:
blocks.rename(inplace=True, columns={'block': 'BLOCK', 'dim': 'DIM', 'logic': 'LOGIC'})
blocks.head()

,BLOCK,DIM,LOGIC,geometry
0,13,"2,5","rows,snake","POLYGON ((-87.62584 41.87433, -87.62581 41.873..."
1,12,"2,5","rows,snake","POLYGON ((-87.62588 41.87562, -87.62585 41.874..."
2,9,"2,5","rows,snake","POLYGON ((-87.62588 41.87693, -87.62585 41.875..."
3,8,"2,5","rows,snake","POLYGON ((-87.62590 41.87818, -87.62588 41.877..."
4,5,"2,5","rows,snake","POLYGON ((-87.62588 41.87944, -87.62585 41.878..."


In [26]:
blocks_with_quarters = gpd.sjoin(blocks, quarters, how="left", predicate='within')

In [27]:
ot = original_town.loc[0, 'geometry']

def in_original_town(geometry):
    return geometry.intersects(ot)

In [28]:
blocks_with_quarters['ORIGINALTOWN'] = blocks_with_quarters['geometry'].apply(in_original_town)

In [29]:
blocks_with_quarters[blocks_with_quarters['ORIGINALTOWN']]

,BLOCK,DIM,LOGIC,geometry,index_right,OBJECTID,TOWNSHIP,RANGE,SECTN,QUARTERSEC,IBL,FRACL,SHAPE_Length,SHAPE_Area,ORIGINALTOWN
86,42,"4,2","rows,snake","POLYGON ((-87.63556 41.88408, -87.63680 41.884...",3865.0,3866.0,39.0,14.0,9,S.E.1/4,0,0.0,10635.834476,7.069399e+06,True
87,41,"4,2","rows,snake","POLYGON ((-87.63408 41.88409, -87.63532 41.884...",3865.0,3866.0,39.0,14.0,9,S.E.1/4,0,0.0,10635.834476,7.069399e+06,True
88,40,"4,2","rows,snake","POLYGON ((-87.63250 41.88416, -87.63374 41.884...",3865.0,3866.0,39.0,14.0,9,S.E.1/4,0,0.0,10635.834476,7.069399e+06,True
89,53,"4,2","rows,snake","POLYGON ((-87.63551 41.88294, -87.63675 41.882...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
90,54,"4,2","rows,snake","POLYGON ((-87.63395 41.88296, -87.63519 41.882...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,59,"2,3","rows,snake","POLYGON ((-87.64562 41.88868, -87.64558 41.887...",3866.0,3867.0,39.0,14.0,9,S.W.1/4,0,0.0,10479.948969,6.864261e+06,True
501,61,"2,4L","rows,snake","POLYGON ((-87.64733 41.88772, -87.64733 41.886...",3866.0,3867.0,39.0,14.0,9,S.W.1/4,0,0.0,10479.948969,6.864261e+06,True
502,62,"2,4L","rows,snake","POLYGON ((-87.64562 41.88771, -87.64558 41.886...",3866.0,3867.0,39.0,14.0,9,S.W.1/4,0,0.0,10479.948969,6.864261e+06,True
503,64,"2,4L","rows,snake","POLYGON ((-87.64732 41.88640, -87.64729 41.885...",3866.0,3867.0,39.0,14.0,9,S.W.1/4,0,0.0,10479.948969,6.864261e+06,True


In [30]:
len(blocks)

707

In [31]:
len(blocks_with_quarters)

707

In [32]:
for idx, row in blocks_with_quarters.loc[blocks_with_quarters['TOWNSHIP'].isnull()].iterrows():
    mask = quarters['geometry'].apply(lambda x : x.intersects(row['geometry']))
    candidates = quarters.loc[mask]
    
    winner = None
    most_overlap = 0
    
    for idx2, row2 in candidates.iterrows():
        overlap = row2['geometry'].intersection(row['geometry']).area
        if overlap > most_overlap:
            winner = idx2
            most_overlap = overlap
            
    if winner:
        blocks_with_quarters.loc[idx, ['OBJECTID','TOWNSHIP','RANGE','SECTN','QUARTERSEC','IBL','FRACL']] =\
        quarters.loc[winner, ['OBJECTID','TOWNSHIP','RANGE','SECTN','QUARTERSEC','IBL','FRACL']].values

In [33]:
len(blocks_with_quarters)

707

In [34]:
blocks_with_quarters['TOWNSHIP'].isnull().sum()

0

In [35]:
blocks_with_quarters.head()

,BLOCK,DIM,LOGIC,geometry,index_right,OBJECTID,TOWNSHIP,RANGE,SECTN,QUARTERSEC,IBL,FRACL,SHAPE_Length,SHAPE_Area,ORIGINALTOWN
0,13,"2,5","rows,snake","POLYGON ((-87.62584 41.87433, -87.62581 41.873...",1519.0,1520.0,39.0,14.0,15,S.W.1/4,0,0.0,10738.940743,7.142300e+06,False
1,12,"2,5","rows,snake","POLYGON ((-87.62588 41.87562, -87.62585 41.874...",NaN,1488.0,39.0,14.0,15,N.W.1/4,0,0.0,NaN,NaN,False
2,9,"2,5","rows,snake","POLYGON ((-87.62588 41.87693, -87.62585 41.875...",1487.0,1488.0,39.0,14.0,15,N.W.1/4,0,0.0,11216.048053,7.907387e+06,False
3,8,"2,5","rows,snake","POLYGON ((-87.62590 41.87818, -87.62588 41.877...",1487.0,1488.0,39.0,14.0,15,N.W.1/4,0,0.0,11216.048053,7.907387e+06,False
4,5,"2,5","rows,snake","POLYGON ((-87.62588 41.87944, -87.62585 41.878...",1487.0,1488.0,39.0,14.0,15,N.W.1/4,0,0.0,11216.048053,7.907387e+06,False


In [38]:
blocks = []
lots = []
townships = []
ranges = []
sections = []
quartersecs = []
original_towns = []
geometry = []

for idx, row in blocks_with_quarters.loc[(blocks_with_quarters['DIM'] != "")\
                                         &blocks_with_quarters['DIM'].notnull()].iterrows():
    try:
        x,y = map(int, row['DIM'].split(","))
    except:
        x,y = row['DIM'].split(",")
        y = int(y.replace("L", ''))
        x = int(x)
        print(idx)
    total = x * y
    
    logic = row['LOGIC'].split(",")
    
    if logic[0] == 'code':
        total = 9
        w,e = sorted(split_polygon(row['geometry'], 2, 1).geoms, key=cmp_to_key(we))
        e_list = sorted(split_polygon(e, 1, 8).geoms, key=cmp_to_key(ns))
        polygons = [w] + e_list
        
    else:
        starts = 'NE'
        if len(logic) == 3:
            starts = logic[2].upper()

        cols = logic[0] == 'cols' or logic[0] == 'col'
        
        snake_length = x
        if cols:
            snake_length = y
        
        polygons = snake_sort(split_polygon(row['geometry'],x, y).geoms, snake_length, starts, cols)
        
    blocks.extend([row['BLOCK'] for _ in range(total)])
    lots.extend([i for i in range(1, total+1)])
    townships.extend([row['TOWNSHIP'] for _ in range(total)])
    ranges.extend([row['RANGE'] for _ in range(total)])
    sections.extend([row['SECTN'] for _ in range(total)])
    quartersecs.extend([row['QUARTERSEC'] for _ in range(total)])
    original_towns.extend([row['ORIGINALTOWN'] for _ in range(total)])
    geometry.extend(polygons)
    
        

52
501
502
503
504


In [39]:
city_lots = {
    'BLOCK': blocks,
    'LOT': lots,
    'TOWNSHIP': townships,
    'RANGE': ranges,
    'SECTN': sections,
    'QUARTERSEC': quartersecs,
    'ORIGINALTOWN': original_towns
}

In [40]:
city_lots_gdf = gpd.GeoDataFrame(city_lots, geometry=geometry)

In [41]:
city_lots_gdf.explore()

In [42]:
blocks_with_quarters.explore()

In [43]:
blocks_with_quarters.to_file("../data/city_blocks.geojson", driver="GeoJSON")
city_lots_gdf.to_file("../data/city_lots.geojson", driver="GeoJSON")